# Aggregating counts and rates of hospital admissions and make a table

In [19]:
import pandas as pd
from matplotlib import pyplot
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

## 0- functions and read data

In [20]:
def data_process(data, data_hosped):
    #conver counts into int
    data['count_int'] = data['count'].astype(str).replace('\.0', '', regex=True).astype(int)
    data_hosped['count_int'] = data['count_int']
    for row in range(0, len(data_hosped)):
        if data_hosped['count'][row] == 'SM':
            data_hosped['count'][row] = 5
    data_hosped['count_hosped_int'] = data_hosped['count'].astype(str).replace('\.0', '', regex=True).astype(int)
    data_hosped['hosped_rate'] = (data_hosped['count_hosped_int']/data['count_int'])*1000
    #custom order of variables to fit tables of the manuscript
    data_hosped['variable'] = pd.Categorical(data_hosped['variable'], ["age_cat_15_24", "age_cat_25_34", "age_cat_35_44", "age_cat_45_54", "age_cat_55_64", "age_cat_65_74", "age_cat_75_more", "sex_male", "sex_female", "bmi_cat_underweight", "bmi_cat_healthy_weight", "bmi_cat_overweight", "bmi_cat_obese", "bmi_cat_unknown", "ethnicity_white", "ethnicity_non-white", "ethnicity_unknown", "CCI_cat_very_low", "CCI_cat_low", "CCI_cat_medium", "CCI_cat_high", "CCI_cat_very_high", "smoking_status_smoker", "smoking_status_ex_smoker", "smoking_status_never_smoked", "smoking_status_unknown", "imd_very_affluent", "imd_affluent", "imd_medium", "imd_unaffluent", "imd_very_unaffluent", "imd_unknown", "season_spring", "season_summer", "season_autumn", "season_winter", "region_london", "region_north_east", "region_north_west", "region_east", "region_west_midlands", "region_yorkshire", "region_south_east", "region_east_midlands", "region_south_west", "flu_vaccine_yes", "flu_vaccine_no", "antibacterial_brit"])
    data_hosped = data_hosped.sort_values('variable')
    #keep mean values of gp_count and antibacterial_brit
    data_hosped['hosped_rate'] = np.round(data_hosped['hosped_rate'], decimals=1)
    #reset index to prevent issue in calculating total value
    data_hosped = data_hosped.reset_index()
    # add comma after every 3 digits
    data_hosped['count_hosped_int']=data_hosped['count_hosped_int'].astype(int)
    data_hosped['count_hosped_int'] = data_hosped['count_hosped_int'].apply(lambda x : "{:,}".format(x))
    #combine counts and means
    for row in range(0, len(data_hosped)):
        if data_hosped['count'][row] == 5:
            data_hosped['count_hosped_int'][row] = 'SN'
            data_hosped['hosped_rate'][row] = 'SN'    
    #combine conts and rates
    data_hosped['count_rate_hosped'] = data_hosped['count_hosped_int'].astype(str) + ' (' + data_hosped['hosped_rate'].astype(str) + ')'
    #calculate total from sex_male and sex_female
    data_hosped['count'] = data_hosped['count'].astype(str).replace('\.0', '', regex=True).astype(int)
    total = int(data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_male'].index[0]),int(data_hosped.columns.get_loc('count'))]) + int(data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_female'].index[0]),int(data_hosped.columns.get_loc('count'))])
    total = "{:,}".format(total)
    new_row = {'index':'', 'variable':'Total', 'count':'', 'mean':'', 'std':'', 'count_int':'', 'count_hosped_int':'', 'hosped_rate':'', 'count_rate_hosped':total} #count_hosped_int, hosped_rate, count_rate_hosped
    #append row to the dataframe
    data_hosped = data_hosped.append(new_row, ignore_index=True)
    #keep variable names and counts comibed with means
    data_hosped = data_hosped[['variable', 'count_rate_hosped']]
    #custom order of variables to fit tables of the manuscript again
    data_hosped['variable'] = pd.Categorical(data_hosped['variable'], ["Total", "age_cat_15_24", "age_cat_25_34", "age_cat_35_44", "age_cat_45_54", "age_cat_55_64", "age_cat_65_74", "age_cat_75_more", "sex_male", "sex_female", "bmi_cat_underweight", "bmi_cat_healthy_weight", "bmi_cat_overweight", "bmi_cat_obese", "bmi_cat_unknown", "ethnicity_white", "ethnicity_non-white", "ethnicity_unknown", "CCI_cat_very_low", "CCI_cat_low", "CCI_cat_medium", "CCI_cat_high", "CCI_cat_very_high", "smoking_status_smoker", "smoking_status_ex_smoker", "smoking_status_never_smoked", "smoking_status_unknown", "imd_very_affluent", "imd_affluent", "imd_medium", "imd_unaffluent", "imd_very_unaffluent", "imd_unknown", "season_spring", "season_summer", "season_autumn", "season_winter", "region_london", "region_north_east", "region_north_west", "region_east", "region_west_midlands", "region_yorkshire", "region_south_east", "region_east_midlands", "region_south_west", "flu_vaccine_yes", "flu_vaccine_no", "antibacterial_brit"])
    data_hosped = data_hosped.sort_values('variable')

    return data_hosped

In [21]:
summary_table_hosped_lrti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_incdt_hosped.csv')
summary_table_hosped_lrti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_incdt_hosped.csv')
summary_table_hosped_lrti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_prevl_hosped.csv')
summary_table_hosped_lrti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_prevl_hosped.csv')
summary_table_hosped_urti_combined_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_incdt_hosped.csv')
summary_table_hosped_urti_combined_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_incdt_hosped.csv')
summary_table_hosped_urti_combined_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_prevl_hosped.csv')
summary_table_hosped_urti_combined_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_prevl_hosped.csv')
summary_table_hosped_uti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_incdt_hosped.csv')
summary_table_hosped_uti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_incdt_hosped.csv')
summary_table_hosped_uti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_prevl_hosped.csv')
summary_table_hosped_uti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_prevl_hosped.csv')
summary_table_hosped_sinusitis_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_incdt_hosped.csv')
summary_table_hosped_sinusitis_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_incdt_hosped.csv')
summary_table_hosped_sinusitis_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_prevl_hosped.csv')
summary_table_hosped_sinusitis_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_prevl_hosped.csv')
summary_table_hosped_otmedia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_incdt_hosped.csv')
summary_table_hosped_otmedia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_incdt_hosped.csv')
summary_table_hosped_otmedia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_prevl_hosped.csv')
summary_table_hosped_otmedia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_prevl_hosped.csv')
summary_table_hosped_ot_externa_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_incdt_hosped.csv')
summary_table_hosped_ot_externa_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_incdt_hosped.csv')
summary_table_hosped_ot_externa_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_prevl_hosped.csv')
summary_table_hosped_ot_externa_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_prevl_hosped.csv')
summary_table_hosped_pneumonia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_no_abs_incdt_hosped.csv')
summary_table_hosped_pneumonia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_abs_incdt_hosped.csv')
summary_table_hosped_pneumonia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_no_abs_prevl_hosped.csv')
summary_table_hosped_pneumonia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_abs_prevl_hosped.csv')
summary_table_hosped_urti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_incdt_hosped.csv')
summary_table_hosped_urti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_incdt_hosped.csv')
summary_table_hosped_urti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_prevl_hosped.csv')
summary_table_hosped_urti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_prevl_hosped.csv')
summary_table_hosped_cough_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_incdt_hosped.csv')
summary_table_hosped_cough_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_incdt_hosped.csv')
summary_table_hosped_cough_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_prevl_hosped.csv')
summary_table_hosped_cough_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_prevl_hosped.csv')
summary_table_hosped_cough_cold_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_incdt_hosped.csv')
summary_table_hosped_cough_cold_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_incdt_hosped.csv')
summary_table_hosped_cough_cold_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_prevl_hosped.csv')
summary_table_hosped_cough_cold_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_prevl_hosped.csv')
summary_table_hosped_throat_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_incdt_hosped.csv')
summary_table_hosped_throat_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_incdt_hosped.csv')
summary_table_hosped_throat_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_prevl_hosped.csv')
summary_table_hosped_throat_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_prevl_hosped.csv')

In [22]:
summary_table_lrti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_incdt.csv')
summary_table_lrti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_incdt.csv')
summary_table_lrti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_prevl.csv')
summary_table_lrti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_prevl.csv')
summary_table_urti_combined_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_incdt.csv')
summary_table_urti_combined_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_incdt.csv')
summary_table_urti_combined_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_prevl.csv')
summary_table_urti_combined_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_prevl.csv')
summary_table_uti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_incdt.csv')
summary_table_uti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_incdt.csv')
summary_table_uti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_prevl.csv')
summary_table_uti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_prevl.csv')
summary_table_sinusitis_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_incdt.csv')
summary_table_sinusitis_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_incdt.csv')
summary_table_sinusitis_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_prevl.csv')
summary_table_sinusitis_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_prevl.csv')
summary_table_otmedia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_incdt.csv')
summary_table_otmedia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_incdt.csv')
summary_table_otmedia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_prevl.csv')
summary_table_otmedia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_prevl.csv')
summary_table_ot_externa_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_incdt.csv')
summary_table_ot_externa_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_incdt.csv')
summary_table_ot_externa_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_prevl.csv')
summary_table_ot_externa_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_prevl.csv')
summary_table_pneumonia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_no_abs_incdt.csv')
summary_table_pneumonia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_abs_incdt.csv')
summary_table_pneumonia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_no_abs_prevl.csv')
summary_table_pneumonia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_abs_prevl.csv')
summary_table_urti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_incdt.csv')
summary_table_urti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_incdt.csv')
summary_table_urti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_prevl.csv')
summary_table_urti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_prevl.csv')
summary_table_cough_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_incdt.csv')
summary_table_cough_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_incdt.csv')
summary_table_cough_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_prevl.csv')
summary_table_cough_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_prevl.csv')
summary_table_cough_cold_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_incdt.csv')
summary_table_cough_cold_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_incdt.csv')
summary_table_cough_cold_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_prevl.csv')
summary_table_cough_cold_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_prevl.csv')
summary_table_throat_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_incdt.csv')
summary_table_throat_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_incdt.csv')
summary_table_throat_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_prevl.csv')
summary_table_throat_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_prevl.csv')

## 1- process data to make a table of counts and rates, and save it

In [23]:
summary_table_lrti_no_abs_incdt = data_process(summary_table_lrti_no_abs_incdt, summary_table_hosped_lrti_no_abs_incdt)
summary_table_lrti_abs_incdt = data_process(summary_table_lrti_abs_incdt, summary_table_hosped_lrti_abs_incdt)
summary_table_lrti_no_abs_prevl = data_process(summary_table_lrti_no_abs_prevl, summary_table_hosped_lrti_no_abs_prevl)
summary_table_lrti_abs_prevl = data_process(summary_table_lrti_abs_prevl, summary_table_hosped_lrti_no_abs_prevl)
summary_table_urti_combined_no_abs_incdt = data_process(summary_table_urti_combined_no_abs_incdt, summary_table_hosped_urti_combined_no_abs_incdt)
summary_table_urti_combined_abs_incdt = data_process(summary_table_urti_combined_abs_incdt, summary_table_hosped_urti_combined_abs_incdt)
summary_table_urti_combined_no_abs_prevl = data_process(summary_table_urti_combined_no_abs_prevl, summary_table_hosped_urti_combined_no_abs_prevl)
summary_table_urti_combined_abs_prevl = data_process(summary_table_urti_combined_abs_prevl, summary_table_hosped_urti_combined_abs_prevl)
summary_table_uti_no_abs_incdt = data_process(summary_table_uti_no_abs_incdt, summary_table_hosped_uti_no_abs_incdt)
summary_table_uti_abs_incdt = data_process(summary_table_uti_abs_incdt, summary_table_hosped_uti_abs_incdt)
summary_table_uti_no_abs_prevl = data_process(summary_table_uti_no_abs_prevl, summary_table_hosped_uti_no_abs_prevl)
summary_table_uti_abs_prevl = data_process(summary_table_uti_abs_prevl, summary_table_hosped_uti_abs_prevl)
summary_table_sinusitis_no_abs_incdt = data_process(summary_table_sinusitis_no_abs_incdt, summary_table_hosped_sinusitis_no_abs_incdt)
summary_table_sinusitis_abs_incdt = data_process(summary_table_sinusitis_abs_incdt, summary_table_hosped_sinusitis_abs_incdt)
summary_table_sinusitis_no_abs_prevl = data_process(summary_table_sinusitis_no_abs_prevl, summary_table_hosped_sinusitis_no_abs_prevl)
summary_table_sinusitis_abs_prevl = data_process(summary_table_sinusitis_abs_prevl, summary_table_hosped_sinusitis_abs_prevl)
summary_table_otmedia_no_abs_incdt = data_process(summary_table_otmedia_no_abs_incdt, summary_table_hosped_otmedia_no_abs_incdt)
summary_table_otmedia_abs_incdt = data_process(summary_table_otmedia_abs_incdt, summary_table_hosped_otmedia_abs_incdt)
summary_table_otmedia_no_abs_prevl = data_process(summary_table_otmedia_no_abs_prevl, summary_table_hosped_otmedia_no_abs_prevl)
summary_table_otmedia_abs_prevl = data_process(summary_table_otmedia_abs_prevl, summary_table_hosped_otmedia_abs_prevl)
summary_table_ot_externa_no_abs_incdt = data_process(summary_table_ot_externa_no_abs_incdt, summary_table_hosped_ot_externa_no_abs_incdt)
summary_table_ot_externa_abs_incdt = data_process(summary_table_ot_externa_abs_incdt, summary_table_hosped_ot_externa_abs_incdt)
summary_table_ot_externa_no_abs_prevl = data_process(summary_table_ot_externa_no_abs_prevl, summary_table_hosped_ot_externa_no_abs_prevl)
summary_table_ot_externa_abs_prevl = data_process(summary_table_ot_externa_abs_prevl, summary_table_hosped_ot_externa_abs_prevl)
summary_table_pneumonia_no_abs_incdt = data_process(summary_table_pneumonia_no_abs_incdt, summary_table_hosped_pneumonia_no_abs_incdt)
summary_table_pneumonia_abs_incdt = data_process(summary_table_pneumonia_abs_incdt, summary_table_hosped_pneumonia_abs_incdt)
summary_table_pneumonia_no_abs_prevl = data_process(summary_table_pneumonia_no_abs_prevl, summary_table_hosped_pneumonia_no_abs_prevl)
summary_table_pneumonia_abs_prevl = data_process(summary_table_pneumonia_abs_prevl, summary_table_hosped_pneumonia_abs_prevl)
summary_table_urti_no_abs_incdt = data_process(summary_table_urti_no_abs_incdt, summary_table_hosped_urti_no_abs_incdt)
summary_table_urti_abs_incdt = data_process(summary_table_urti_abs_incdt, summary_table_hosped_urti_abs_incdt)
summary_table_urti_no_abs_prevl = data_process(summary_table_urti_no_abs_prevl, summary_table_hosped_urti_no_abs_prevl)
summary_table_urti_abs_prevl = data_process(summary_table_urti_abs_prevl, summary_table_hosped_urti_abs_prevl)
summary_table_cough_no_abs_incdt = data_process(summary_table_cough_no_abs_incdt, summary_table_hosped_cough_no_abs_incdt)
summary_table_cough_abs_incdt = data_process(summary_table_cough_abs_incdt, summary_table_hosped_cough_abs_incdt)
summary_table_cough_no_abs_prevl = data_process(summary_table_cough_no_abs_prevl, summary_table_hosped_cough_no_abs_prevl)
summary_table_cough_abs_prevl = data_process(summary_table_cough_abs_prevl, summary_table_hosped_cough_abs_prevl)
summary_table_cough_cold_no_abs_incdt = data_process(summary_table_cough_cold_no_abs_incdt, summary_table_hosped_cough_cold_no_abs_incdt)
summary_table_cough_cold_abs_incdt = data_process(summary_table_cough_cold_abs_incdt, summary_table_hosped_cough_cold_abs_incdt)
summary_table_cough_cold_no_abs_prevl = data_process(summary_table_cough_cold_no_abs_prevl, summary_table_hosped_cough_cold_no_abs_prevl)
summary_table_cough_cold_abs_prevl = data_process(summary_table_cough_cold_abs_prevl, summary_table_hosped_cough_cold_abs_prevl)
summary_table_throat_no_abs_incdt = data_process(summary_table_throat_no_abs_incdt, summary_table_hosped_throat_no_abs_incdt)
summary_table_throat_abs_incdt = data_process(summary_table_throat_abs_incdt, summary_table_hosped_throat_abs_incdt)
summary_table_throat_no_abs_prevl = data_process(summary_table_throat_no_abs_prevl, summary_table_hosped_throat_no_abs_prevl)
summary_table_throat_abs_prevl = data_process(summary_table_throat_abs_prevl, summary_table_hosped_throat_abs_prevl)

In [24]:
d = {'variable': summary_table_lrti_no_abs_incdt['variable'],
     
     'lrti_no_abs_incdt': summary_table_lrti_no_abs_incdt['count_rate_hosped'], 
     'lrti_abs_incdt': summary_table_lrti_abs_incdt['count_rate_hosped'],
     'lrti_no_abs_prevl': summary_table_lrti_no_abs_prevl['count_rate_hosped'], 
     'lrti_abs_prevl': summary_table_lrti_abs_prevl['count_rate_hosped'],
     
     'urti_combined_no_abs_incdt': summary_table_urti_combined_no_abs_incdt['count_rate_hosped'], 
     'urti_combined_abs_incdt': summary_table_urti_combined_abs_incdt['count_rate_hosped'],
     'urti_combined_no_abs_prevl': summary_table_urti_combined_no_abs_prevl['count_rate_hosped'], 
     'urti_combined_abs_prevl': summary_table_urti_combined_abs_prevl['count_rate_hosped'],
     
     'uti_no_abs_incdt': summary_table_uti_no_abs_incdt['count_rate_hosped'], 
     'uti_abs_incdt': summary_table_uti_abs_incdt['count_rate_hosped'],
     'uti_no_abs_prevl': summary_table_uti_no_abs_prevl['count_rate_hosped'], 
     'uti_abs_prevl': summary_table_uti_abs_prevl['count_rate_hosped'],
     
     'sinusitis_no_abs_incdt': summary_table_sinusitis_no_abs_incdt['count_rate_hosped'], 
     'sinusitis_abs_incdt': summary_table_sinusitis_abs_incdt['count_rate_hosped'],
     'sinusitis_no_abs_prevl': summary_table_sinusitis_no_abs_prevl['count_rate_hosped'], 
     'sinusitis_abs_prevl': summary_table_sinusitis_abs_prevl['count_rate_hosped'],
     
     'otmedia_no_abs_incdt': summary_table_otmedia_no_abs_incdt['count_rate_hosped'], 
     'otmedia_abs_incdt': summary_table_otmedia_abs_incdt['count_rate_hosped'],
     'otmedia_no_abs_prevl': summary_table_otmedia_no_abs_prevl['count_rate_hosped'], 
     'otmedia_abs_prevl': summary_table_otmedia_abs_prevl['count_rate_hosped'],
     
     'ot_externa_no_abs_incdt': summary_table_ot_externa_no_abs_incdt['count_rate_hosped'], 
     'ot_externa_abs_incdt': summary_table_ot_externa_abs_incdt['count_rate_hosped'],
     'ot_externa_no_abs_prevl': summary_table_ot_externa_no_abs_prevl['count_rate_hosped'], 
     'ot_externa_abs_prevl': summary_table_ot_externa_abs_prevl['count_rate_hosped'],

     'pneumonia_no_abs_incdt': summary_table_pneumonia_no_abs_incdt['count_rate_hosped'], 
     'pneumonia_abs_incdt': summary_table_pneumonia_abs_incdt['count_rate_hosped'],
     'pneumonia_no_abs_prevl': summary_table_pneumonia_no_abs_prevl['count_rate_hosped'], 
     'pneumonia_abs_prevl': summary_table_pneumonia_abs_prevl['count_rate_hosped'],

     'urti_no_abs_incdt': summary_table_urti_no_abs_incdt['count_rate_hosped'], 
     'urti_abs_incdt': summary_table_urti_abs_incdt['count_rate_hosped'],
     'urti_no_abs_prevl': summary_table_urti_no_abs_prevl['count_rate_hosped'], 
     'urti_abs_prevl': summary_table_urti_abs_prevl['count_rate_hosped'],

     'cough_no_abs_incdt': summary_table_cough_no_abs_incdt['count_rate_hosped'], 
     'cough_abs_incdt': summary_table_cough_abs_incdt['count_rate_hosped'],
     'cough_no_abs_prevl': summary_table_cough_no_abs_prevl['count_rate_hosped'], 
     'cough_abs_prevl': summary_table_cough_abs_prevl['count_rate_hosped'],

     'cough_cold_no_abs_incdt': summary_table_cough_cold_no_abs_incdt['count_rate_hosped'], 
     'cough_cold_abs_incdt': summary_table_cough_cold_abs_incdt['count_rate_hosped'],
     'cough_cold_no_abs_prevl': summary_table_cough_cold_no_abs_prevl['count_rate_hosped'], 
     'cough_cold_abs_prevl': summary_table_cough_cold_abs_prevl['count_rate_hosped'],

     'throat_no_abs_incdt': summary_table_throat_no_abs_incdt['count_rate_hosped'], 
     'throat_abs_incdt': summary_table_throat_abs_incdt['count_rate_hosped'],
     'throat_no_abs_prevl': summary_table_throat_no_abs_prevl['count_rate_hosped'], 
     'throat_abs_prevl': summary_table_throat_abs_prevl['count_rate_hosped'],
    }
data = pd.DataFrame(data=d)

data.to_csv(f'../output/aggregate_hosp_table/counts_rates_table_hosped_all_infections.csv', index=False)
data

,variable,lrti_no_abs_incdt,lrti_abs_incdt,lrti_no_abs_prevl,lrti_abs_prevl,urti_combined_no_abs_incdt,urti_combined_abs_incdt,urti_combined_no_abs_prevl,urti_combined_abs_prevl,uti_no_abs_incdt,...,cough_no_abs_prevl,cough_abs_prevl,cough_cold_no_abs_incdt,cough_cold_abs_incdt,cough_cold_no_abs_prevl,cough_cold_abs_prevl,throat_no_abs_incdt,throat_abs_incdt,throat_no_abs_prevl,throat_abs_prevl
0,age_cat_15_24,90 (7.4),SN (SN),10 (7.4),10 (66.7),570 (6.9),60 (6.5),65 (7.0),SN (SN),145 (7.1),...,10 (7.4),SN (SN),80 (6.6),10 (7.2),15 (10.5),SN (SN),145 (7.1),15 (6.5),10 (4.4),SN (SN)
1,age_cat_25_34,85 (6.9),10 (7.1),10 (7.1),10 (74.1),635 (7.8),60 (6.7),75 (8.4),SN (SN),145 (7.0),...,15 (10.9),SN (SN),100 (8.1),10 (7.3),10 (7.2),SN (SN),155 (7.7),15 (6.8),20 (9.1),SN (SN)
2,age_cat_35_44,95 (7.7),10 (7.2),10 (7.0),10 (64.5),625 (7.7),65 (7.2),65 (7.2),10 (10.6),155 (7.5),...,10 (7.3),SN (SN),90 (7.4),10 (7.5),SN (SN),SN (SN),190 (9.3),20 (9.0),15 (6.6),SN (SN)
3,age_cat_45_54,80 (6.6),15 (10.7),10 (7.2),10 (57.1),600 (7.3),80 (8.9),75 (8.2),SN (SN),140 (6.8),...,10 (6.9),SN (SN),85 (6.9),15 (11.1),15 (10.5),SN (SN),170 (8.2),25 (11.2),20 (8.7),SN (SN)
4,age_cat_55_64,85 (6.8),10 (7.1),10 (7.4),10 (74.1),605 (7.4),75 (8.2),80 (8.8),SN (SN),165 (8.3),...,10 (7.3),SN (SN),100 (8.2),10 (7.6),10 (7.1),SN (SN),135 (6.6),10 (4.2),25 (11.2),SN (SN)
5,age_cat_65_74,95 (7.7),10 (7.3),10 (7.5),10 (62.5),595 (7.3),70 (7.7),70 (7.8),SN (SN),160 (7.7),...,10 (7.3),SN (SN),80 (6.5),10 (7.3),15 (10.6),SN (SN),135 (6.6),30 (12.9),10 (4.5),10 (37.0)
6,age_cat_75_more,90 (7.4),SN (SN),10 (7.3),10 (69.0),620 (7.5),75 (8.2),55 (6.1),SN (SN),160 (7.9),...,10 (7.4),SN (SN),95 (7.7),15 (11.0),SN (SN),SN (SN),160 (7.8),20 (8.9),10 (4.4),SN (SN)
7,sex_male,275 (6.6),20 (4.1),30 (6.4),30 (59.4),"2,120 (7.5)",230 (7.3),240 (7.7),20 (5.8),515 (7.3),...,30 (6.4),SN (SN),315 (7.4),40 (8.5),30 (6.3),SN (SN),570 (8.1),60 (7.6),55 (7.1),SN (SN)
8,sex_female,345 (7.8),45 (9.2),35 (7.1),35 (63.1),"2,135 (7.3)",250 (7.8),240 (7.4),20 (5.6),565 (7.7),...,40 (8.1),SN (SN),315 (7.3),45 (9.3),40 (8.0),SN (SN),525 (7.2),70 (8.7),55 (6.8),SN (SN)
9,bmi_cat_underweight,25 (8.9),SN (SN),SN (SN),SN (SN),150 (8.1),15 (7.2),15 (7.4),SN (SN),35 (7.3),...,SN (SN),SN (SN),20 (7.3),SN (SN),SN (SN),SN (SN),35 (7.2),SN (SN),SN (SN),SN (SN)
